In [1]:
import pandas as pd
pd.options.display.max_columns = 140
import numpy as np

In [2]:
terror = pd.read_excel(r"C:\Users\nfa\Dropbox\Ed\Ed Uni work\EME\Data\Original Data\globalterrorismdb_0617dist.xlsx")

In [3]:
terror.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaT,58,Dominican Republic,2,Central America & Caribbean,NaN,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,58.0,Dominican Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MANO-D,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaT,130,Mexico,1,North America,NaN,Mexico city,19.432608,-99.133207,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,7,Government (Diplomatic),45.0,"Diplomatic Personnel (outside of embassy, cons...",Belgian Ambassador Daughter,"Nadine Chaval, daughter",21.0,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23rd of September Communist League,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,NaN,Mexico,1.0,800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaT,160,Philippines,5,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,4.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,1,Assassination,NaN,NaN,NaN,NaN,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,Voice of America,Employee,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaT,78,Greece,8,Western Europe,Attica,Athens,37.983773,23.728157,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,7,Government (Diplomatic),46.0,Embassy/Consulate,NaN,U.S. Embassy,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives/Bombs/Dynamite,16.0,Unknown Explosive Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosive,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,

In [4]:
keep_these_variables = ['eventid',
                        'iyear',
                        'imonth',
                       'iday',
                        'country_txt',
                       'country',
                       'nkill',
                       'nwound',
                        'propvalue',
                       'city',
                       'success',
                       'suicide',
                        'gname',
                       'attacktype1',
                       'attacktype1_txt',
                       'targtype1',
                       'targtype1_txt',
                       'weaptype1',
                       'weaptype1_txt',
                        'attacktype2',
                       'attacktype2_txt',
                       'targtype2',
                       'targtype2_txt',
                       'weaptype2',
                       'weaptype2_txt',
                        'attacktype3',
                       'attacktype3_txt',
                       'targtype3',
                       'targtype3_txt',
                       'weaptype3',
                       'weaptype3_txt'
                       'crit1',
                       'crit2',
                       'crit3',
                       'doubtterr']

def smaller_data(country, df, variables = keep_these_variables, merged=True, verbose=False):
    """
    The purpose of this function is:
    A) Only keep the variables we want from the original Maryland GTD source since it contains some 100+ variables.
    B) To clean the data to a degree, this means creating a usable datetime column, dropping missing observation and recoding
    some missing values as 0 as they're the same for our purpose.
    C) To select the country we want data from.
    """
    # Only keep the variables we want in the dataset
    small_data = df.loc[:, variables].copy()
    
    #Choosing a specific country
    small_data = small_data[small_data['country_txt'] == country]
    
    #Adding an incident number so groupby(date) will give distinct attacks that day, as well as making our count() function
    #easier below
    small_data['incident'] = 1
    
    #Counting observations where dates are unknown or approximated
    missing_month_observations = small_data['incident'][small_data['imonth'] == 0].count()
    missing_day_observations = small_data['incident'][small_data['iday'] == 0].count()
    
    #Removing observations where day & month is unknown - TODO: Check that these are MCAR (Missing Completely at Random)
    small_data = small_data[small_data['imonth'] > 0]
    small_data = small_data[small_data['iday'] > 0]
    
    #Creating a datetime column from the year, month and day data given
    small_data['iyear'] = small_data['iyear'].astype(str)
    small_data['imonth'] = small_data['imonth'].astype(str)
    small_data['iday'] = small_data['iday'].astype(int).astype(str)
    small_data['YMD'] = (small_data['iyear'] + '/' + small_data['imonth'] + '/' + small_data['iday'])
    small_data['Date'] = pd.to_datetime(small_data['YMD'], format= '%Y/%m/%d')
    
    #Filling in N/A values with 0
    small_data['propvalue'] = small_data['propvalue'].fillna(value=0)
    small_data['nkill'] = small_data['nkill'].fillna(value=0)
    small_data['nwound'] = small_data['nwound'].fillna(value=0)
    
    #N.B. Property data has unknown coded as -99
    small_data['propvalue'] = small_data['propvalue'].replace(to_replace=-99, value = 0)
    
    #Currently multiple attacks on each day are distinct, this will merge them    
    grouped_data = small_data.groupby( by= 'Date', as_index=False).agg({'nkill' : np.sum,
                                                                   'nwound': np.sum,
                                                                   'propvalue': np.sum,
                                                                   'incident': np.sum,
                                                                       'eventid': 'first',
                                                                    'iyear': 'first',
                                                                    'imonth': 'first',
                                                                   'iday' : 'first',
                                                                    'country_txt': 'first',
                                                                   'country': 'first'})
    
    #sending grouped data as default and no missing info
    if merged == True :
        final_data = grouped_data
    else:
        final_data = small_data
    if verbose == True :
        return final_data, missing_month_observations, missing_day_observations
    else:
        return final_data
       


In [5]:
def weekend_adjuster(data, merged= True):
    """
    The purpose of this function is to move observations from a weekend to a Monday. This is because if an attack occurs on a 
    weekend, since the markets are closed, the index/exchange rate won't react till Monday when markets reopen.
    """
    
    from pandas.tseries.offsets import DateOffset
    
    #Finding the Date column index number
    date_col = data.columns.get_loc('Date') 
    
    #Creating indicators if an attack occurs on Saturday or Sunday
    data['Sat_indicator'] = (data['Date'].dt.weekday == 5)
    data['Sun_indicator'] = (data['Date'].dt.weekday == 6)
    
    #Resetting the index so that the iloc identifier corresponds to the correct observation
    #There's probably a more elegant way to do this
    data_reset_index = data.reset_index()
    
    #Creating a list that contains the iloc row identifier for every weekend attack in the dataset
    saturdays = list(data_reset_index[data_reset_index['Sat_indicator'] == 1].index)
    sundays = list(data_reset_index[data_reset_index['Sun_indicator'] == 1].index)
    
    #For every sat we add two to the observed Date (we don't change iday for reference purposes)
    for event in saturdays:
        offset_date = (data.iloc[event, date_col] + DateOffset(days=2))
        data.iloc[event, date_col] = offset_date
    #Now Sunday:
    for event in sundays:
        offset_date = (data.iloc[event, date_col] + DateOffset(days=1))
        data.iloc[event, date_col] = offset_date
        
    #Finally, we need to re-aggregate any attacks that now occur on Monday that didn't before    
    if merged == True :
        grouped_data = data.groupby( by= 'Date', as_index=False).agg({'nkill' : np.sum,
                                                                   'nwound': np.sum,
                                                                   'propvalue': np.sum,
                                                                   'incident': np.sum,
                                                                       'eventid': 'first',
                                                                    'iyear': 'first',
                                                                    'imonth': 'first',
                                                                   'iday' : 'first',
                                                                    'country_txt': 'first',
                                                                   'country': 'first'})
        return grouped_data
    else:
        return data
        

In [6]:
def simple_event_creator(country, df, save = False):
    '''
    Combines the events.
    '''
    events = weekend_adjuster(smaller_data(country = country,
                                          df = df))
    if save == True:
        init_path = "C://Users//nfa//Dropbox//Ed//Ed Uni work//EME//Data//Clean Data//Terror//"
        event_path = country + '.xls'
        full_path = init_path + event_path
        events.to_excel(full_path)
    return events

In [7]:
UK_df, months, days = smaller_data(country='United Kingdom', df=terror, merged=False, variables=keep_these_variables, verbose=True)
print( months, days)
UK_terror_cleaned = weekend_adjuster(UK_df)
#UK_terror_cleaned.to_excel(r'E:\EME Data\Clean Data\Terror\UK_Terror_Clean2.xlsx')

0 27


In [8]:
US_events = simple_event_creator('United States', terror)
GER_events = simple_event_creator('Germany', terror)
AUS_events = simple_event_creator('Australia', terror)
ISRAEL_events = simple_event_creator('Israel', terror)
UK_events = simple_event_creator('United Kingdom', terror)

In [9]:
US_events

,Date,nkill,nwound,propvalue,incident,eventid,iyear,imonth,iday,country_txt,country
0,1970-01-01,0.0,0.0,0.0,2,197001010002,1970,1,1,United States,217
1,1970-01-02,0.0,0.0,82500.0,2,197001020002,1970,1,2,United States,217
2,1970-01-05,0.0,0.0,0.0,1,197001030001,1970,1,3,United States,217
3,1970-01-06,0.0,0.0,305.0,1,197001060001,1970,1,6,United States,217
4,1970-01-09,0.0,0.0,2000000.0,2,197001090001,1970,1,9,United States,217
5,1970-01-12,0.0,0.0,0.0,2,197001120001,1970,1,12,United States,217
6,1970-01-13,0.0,0.0,17000.0,1,197001130001,1970,1,13,United States,217
7,1970-01-14,0.0,1.0,0.0,1,197001140001,1970,1,14,United States,217
8,1970-01-19,0.0,0.0,2200.0,3,197001190002,1970,1,19,United States,217
9,1970-01-22,0.0,0.0,0.0,1,197001220002,1970,1,22,United States,217
